In [23]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
from scipy.interpolate import griddata
from IPython.display import display
from fem_pos_processing import fem_analysis as fem

## `get_dir()`

In [24]:
def get_dir(name):
    """
    Gera o caminho completo para salvar o arquivo e garante que o diretório exista.
    
    Parâmetros:
        name (str): Nome do arquivo com o caminho relativo a partir do diretório base.

    Retorna:
        str: Caminho completo para o arquivo, tratado para espaços e barras.
    """
    base_dir = os.getcwd()  # Obtém o diretório atual
    filepath = os.path.normpath(os.path.join(base_dir, name))  # Normaliza o caminho completo
    directory = os.path.dirname(filepath)  # Obtém o diretório onde o arquivo será salvo

    # Garante que o diretório exista
    os.makedirs(directory, exist_ok=True)
    
    # Retorna o caminho normalizado
    return filepath

# `structured_data()`

In [29]:
def structured_data(mesh_data):
    # Extraindo as coordenadas globais dos nós (x, y)
    try:
        xg = [node['xg'][0] for node in mesh_data['nodes'].values()]
        yg = [node['xg'][1] for node in mesh_data['nodes'].values()]
    except KeyError as e:
        raise ValueError(f"Erro ao acessar as coordenadas globais: {e}")

    # Extraindo a matriz de conectividade
    try:
        conn = [cell['conn'] for cell in mesh_data['cell'].values()]
        # Ajusta índice para 0-based
        conn_py = [[node - 1 for node in nodes[:3]] for nodes in conn]  
    except KeyError as e:
        raise ValueError(f"Erro ao acessar a conectividade: {e}")
    
    return xg, yg, conn_py

# `plot_mesh()`

In [30]:
def plot_mesh(FINITE_ELEMENT, mesh_data, domain='Entire', numbering=False, show_edges=False):
    ElementType, ElementOrder = FINITE_ELEMENT

    # Estruturando os dados da malha
    nodes_data = mesh_data['nodes']
    
    # Extraindo as coordenadas globais dos nós (x, y) e a matriz de conectividade
    xg, yg, conn_py = structured_data(mesh_data)

    plt.figure(figsize=(8, 6))
    if ElementType == "Triangle":
        # Plotando a malha de elementos finitos
        plt.triplot(xg, yg, conn_py, color='gray')  
        
        # Plotando as arestas dos elementos
        if show_edges:
            for key, edge in mesh_data['edges'].items():
                # Coordenadas dos nós inicial e final
                x_st = nodes_data[edge[0]]['xg'][0]
                y_st = nodes_data[edge[0]]['xg'][1]
                x_nd = nodes_data[edge[1]]['xg'][0]
                y_nd = nodes_data[edge[1]]['xg'][1]
                
                # Ponto médio da aresta
                x_mid, y_mid = (x_st + x_nd) / 2, (y_st + y_nd) / 2
                
                # Vetor da seta (a partir do ponto médio)
                dx, dy = (x_nd - x_st) * 0.2, (y_nd - y_st) * 0.2  
                
                # Adicionando uma seta no meio da aresta
                plt.arrow(x_mid, y_mid, dx, dy, head_width=0.015, head_length=0.05,
                    fc='blue', ec='blue', length_includes_head=True)

                # Adicionando os números das arestas
                plt.scatter(x_mid, y_mid, marker='s', color='white', edgecolor='black', s=160, zorder=1)                
                plt.text(x_mid, y_mid, key, color='blue', fontsize=8, ha='center', va='center')

        # Adicionando números dos nós e elementos
        if numbering:            
            for key, node in nodes_data.items():
                x, y = node['xg'][0], node['xg'][1]
                plt.scatter(x, y, color='white', edgecolor='black', s=180)
                plt.text(x, y, str(key), color='red', fontsize=8, ha='center', va='center')

            for key, edge in mesh_data['cell'].items():
                x_c = np.mean([nodes_data[node]['xg'][0] for node in edge['conn']])
                y_c = np.mean([nodes_data[node]['xg'][1] for node in edge['conn']])
                plt.text(x_c, y_c, str(key), fontweight='bold',
                            color='black', fontsize=9, ha='center', va='center')
                
    if not numbering:
        plt.scatter(xg, yg, color='black', s=1, zorder=3)   
        
    # Ajustando rótulos e layout
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.axis('equal')
    plt.tight_layout()
    
    # Salvando o arquivo no formato SVG
    filepath = get_dir(f"pre_processing/pictures/meshed_domain_{ElementType}{ElementOrder}_{domain}Domain.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

# `plot_meshed_shape_func()`

In [31]:
def plot_meshed_shape_func(FINITE_ELEMENT, mesh_data):

    """
    Plota a malha com a possibilidade de mostrar a função de forma vetorial \varphi_1 sobre os elementos.
    """
    type, order = FINITE_ELEMENT
    plot_phi1 = True  # Plotar a função de forma vetorial \varphi_1

    # Ajustar o tamanho dos pontos dependendo da ordem do elemento
    size_dot = 5 if order == 1 else 2

    # Extraindo as coordenadas globais dos nós (x, y) e a matriz de conectividade
    xg, yg, conn_py = structured_data(mesh_data)

    plt.figure(figsize=(10, 8))
    if type == "Triangle":
        plt.triplot(xg, yg, conn_py, color='gray')  
        
        if plot_phi1:
            for cell in [conn_py[0]]:  # Apenas o primeiro elemento
                # Coordenadas dos vértices do elemento
                vertices = [(xg[v], yg[v]) for v in cell]
                
                # Pontos internos do triângulo para plotar \varphi_1
                xi = np.linspace(0, 1, 8)
                eta = np.linspace(0, 1, 8)
                xi_grid, eta_grid = np.meshgrid(xi, eta)
                inside_triangle = xi_grid + eta_grid <= 1
                xi_points = xi_grid[inside_triangle]
                eta_points = eta_grid[inside_triangle]
                
                for xi, eta in zip(xi_points, eta_points):
                    # Calcular o vetor da função de forma \varphi_1
                    N1 = np.array([[1 - eta], [xi]])  # Definição da função \varphi_1
                    
                    # Coordenadas globais do ponto
                    x = (1 - eta - xi) * vertices[0][0] + xi * vertices[1][0] + eta * vertices[2][0]
                    y = (1 - eta - xi) * vertices[0][1] + xi * vertices[1][1] + eta * vertices[2][1]        
                    
                    # Plotar o vetor \varphi_1 no ponto
                    plt.quiver(x, y, N1[0, 0], N1[1, 0], angles='xy', scale_units='xy', scale=10, color='green', zorder=2, width=3E-3)

    # Ajustando rótulos e layout
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()

# `fem_solution()`

In [34]:
def fem_solution(FINITE_ELEMENT, mesh_data, uh, Griddata=False, domain='Entire', type='real'):
    # Extração de informações sobre o elemento finito
    ElementType, ElementOrder = FINITE_ELEMENT
    
    # Conversão do dicionário de soluções para lista
    if type == 'real':
        uh = np.real(list(uh.values()))
    elif type == 'imag':
        uh = np.imag(list(uh.values()))
    elif type == 'abs':
        uh = np.abs(list(uh.values()))
    else:
        raise ValueError("Tipo de solução inválida. Use 'real', 'imag' ou 'abs'.")

    # Extraindo as coordenadas globais dos nós (x, y) e a matriz de conectividade
    xg, yg, conn = structured_data(mesh_data)

    # Plotar a solução aproximada u(x, y) sobre o domínio
    plt.figure(figsize=(8, 6))
    
    # Interpolação cúbica para elementos triangulares
    if Griddata:
        # Gerar uma grade regular para interpolação
        nodes = np.column_stack((xg, yg))
        grid_x, grid_y = np.mgrid[0:1:500j, 0:1:500j]
        grid_u = griddata(nodes, uh, (grid_x, grid_y), method='cubic')
        plt.colorbar(plt.contourf(grid_x, grid_y, grid_u, levels=100, cmap='viridis'))
        filepath = get_dir(f"pos_processing/pictures/fem_solution_griddata_{ElementType}{ElementOrder}_{domain}Domain.svg")

    # Triangulação para elementos triangulares
    else:
        if ElementType == "Triangle":
            triangulation = Triangulation(xg, yg, conn)
        else:
            raise ValueError("Apenas elementos triangulares ('Triangle') são suportados.")

        # Plotar a solução aproximada u(x, y) sobre os elementos
        plt.tricontourf(triangulation, uh, cmap='viridis')

        # Plotar a malha de elementos finitos
        plt.triplot(triangulation, color='gray', alpha=0.5)
        plt.colorbar(label=r'$u(x, y)$')
        filepath = get_dir(f"pos_processing/pictures/fem_solution_{ElementType}{ElementOrder}_{domain}Domain.svg")
    
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.tight_layout()
    plt.axis('equal')

    # Salvando o gráfico
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

# `plot_convergence()`

In [36]:
def plot_convergence(mesh_sizes, error_results, type, points_coord=False):
    plt.figure(figsize=(10, 8))
    
    for element_type, errors in error_results.items():
        # Plotando os pontos
        plt.loglog(mesh_sizes, errors, marker='o', linestyle='-', 
                   label=f'{element_type[0]} Order {element_type[1]}')
        
        # Calculando as inclinações
        slopes = []
        for i in range(1, len(mesh_sizes)):
            delta_x = np.log10(mesh_sizes[i]) - np.log10(mesh_sizes[i-1])
            delta_y = np.log10(errors[i]) - np.log10(errors[i-1])
            slope = delta_y / delta_x
            slopes.append(slope)
            
            # Adicionando o valor da inclinação no meio do segmento
            mid_x = np.sqrt(mesh_sizes[i] * mesh_sizes[i-1])
            mid_y = np.sqrt(errors[i] * errors[i-1])
            plt.text(mid_x, mid_y, f"{slope:.4f}", fontsize=8, 
                     ha='center', va='bottom', color='blue')

        # Adicionando os valores (x, y) ao lado de cada ponto
        if points_coord:
            for x, y in zip(mesh_sizes, errors):
                plt.text(float(x), float(y), f"({x:.4e},\n{y:.4e})", 
                         fontsize=7, ha='right', va='bottom')

    plt.xlabel(r'Element size $h$')
    plt.ylabel(f'{type} norm of the error')
    plt.legend(title="Element Types")
    plt.grid(which='both', linestyle='--', linewidth=0.5)  
    plt.tight_layout()

    # Salvando o arquivo no formato SVG
    filepath = get_dir(f"pos_processing/pictures/convergence_{type}.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

# `matrix2table()`

In [ ]:
def matrix2table(matrix, title):
    """
    Converte matrizes elementares em uma tabela para exibição com título integrado em formato LaTeX.
    
    Parameters:
    - matrix: lista de arrays 2D.
    - title: string, título a ser exibido acima da tabela.
    """
    # Verificar se matrix é uma lista de arrays 2D
    if isinstance(matrix, list):     
      # Converter para DataFrame
      data = {f"Matrix {i+1}": ka.flatten() for i, ka in enumerate(matrix)}
      
      # Transpor para ajustar o layout
      df = pd.DataFrame(data).T
      
      # Renomear colunas para mais clareza
      df.columns = [f"Entry {i+1}" for i in range(df.shape[1])]
    
    elif isinstance(matrix, np.ndarray):
      # Converter para DataFrame
      df = pd.DataFrame(matrix)
      
      # Renomear linhas e colunas para maior clareza
      df.index = [f"{i+1}" for i in range(df.shape[0])]
      df.columns = [f"{i+1}" for i in range(df.shape[1])]
         
    # Aplicar formatação de números para notação científica com 2 casas decimais
    df = df.apply(lambda x: x.map(lambda v: "{:.3e}".format(v)))
    
    # Adicionar título como legenda ao DataFrame estilizado
    styled_df = df.style.set_caption(title).set_table_styles(
        [{'selector': 'caption',
          'props': [('caption-side', 'top'), ('text-align', 'center'), 
                    ('font-size', '18px'), ('font-weight', 'bold')]}]
    )
    
    # Exibir a tabela
    display(styled_df)